In [1]:
import sys
import xmltodict
from collections import OrderedDict
import imageio.v3 as iio
from pathlib import Path
from copy import deepcopy

from turbx import REPO_PATH, log
from turbx.data import find_files

# max number of frames per video
MAX_LEN=300
FPS=10.0

[DEBUG] 11/04/2022 01:00:39PM: MainProcess: __init__.py - Loaded logging config file: /Users/nowa201/Code/fish_detector/turbx/src/turbx/logging.yaml


In [2]:
label_path = f"{REPO_PATH}/data/labels/cvat-video-1.1/test"
video_path = f"{REPO_PATH}/data/mp4/test"
label_out_path = Path(f"{REPO_PATH}/data/labels/cvat-video-1.1/batched_test")
video_out_path = Path(f"{REPO_PATH}/data/mp4/batched_test")
label_out_path.mkdir(exist_ok=True)
video_out_path.mkdir(exist_ok=True)

# collect list of all files in paths
video_files = find_files(video_path)
label_files = find_files(label_path, file_type=".xml")

log.debug(video_files[:3])
log.debug(label_files[:3])

[DEBUG] 11/04/2022 01:00:39PM: MainProcess: 332178680.py - [PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/mp4/test/2010-09-09_041500_HF_aft.mp4'), PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/mp4/test/2010-09-09_053000_HF_aft.mp4'), PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/mp4/test/2010-09-09_051500_HF_aft.mp4')]
[DEBUG] 11/04/2022 01:00:39PM: MainProcess: 332178680.py - [PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/labels/cvat-video-1.1/test/48.xml'), PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/labels/cvat-video-1.1/test/49.xml'), PosixPath('/Users/nowa201/Code/fish_detector/turbx/data/labels/cvat-video-1.1/test/39.xml')]


In [3]:
# loop through label files, match with video, batch into subfiles

def split_tracks(track_list, start_frame, end_frame):
    if not isinstance(track_list, list):
        track_list = [track_list]

    otrack_list = []
    #otrack_dict = {'@id': None, '@label': None, '@source': None, 'box': []}
    for track in track_list:
        otrack_dict = OrderedDict({'@id': track['@id'], '@label': track['@label'], '@source': track['@source'], 'box': []})
        for box in track['box']:
            box_frame_idx = int(box['@frame'])
            if start_frame <= box_frame_idx <= end_frame:
                otrack_dict['box'].append(box)
        if len(otrack_dict['box']) > 0:
            otrack_list.append(otrack_dict.copy())

    return otrack_list


# loop through every label file
for label_file in label_files:
    # vars
    video_file = Path()
    sub_video_counter = 0

    with open(str(label_file), 'rb') as f:
        label_xml_dict = xmltodict.parse(f, encoding='utf-8', xml_attribs=True)
    video_info_dict = label_xml_dict['annotations']['meta']['task']
    video_size = int(video_info_dict['size'])
    log.info(f"Splitting video: {video_info_dict['id']}")

    # verify video with name exists
    for v in video_files:
        if video_info_dict["name"] in v.name:
            video_file = v
            break
    # go to next iteration if video didn't exist
    if not video_file.exists():
        continue

    # loop through greater video splitting into sub_videos
    video = iio.imread(str(video_file))
    while (sub_video_counter*MAX_LEN) < video_size:
        log.info(f"Creating sub-video: {video_info_dict['id']}/{sub_video_counter:04}")

        # TODO: double counting first/last frame of split videos!
        # - Fixed?
        # [0, 299] -> size 0
        start_frame = sub_video_counter*MAX_LEN # 0
        end_frame = ((sub_video_counter+1)*MAX_LEN)-1 # 299
        end_frame = end_frame if end_frame < video_size else (video_size-1)
        sub_video_label = deepcopy(label_xml_dict)

        # E.g: name-0001.mp4
        sub_video_file = Path(f"{video_out_path}/{video_file.stem}-{sub_video_counter:04}{video_file.suffix}")
        # write start_frame to end from video_file
        writer = iio.imwrite(sub_video_file, video[start_frame:end_frame, ...], fps=FPS) # so much faster than writing each frame

        # modify label file for sub-video
        # split tracks based on those in sub-video (split track if necessary - can keep track id?)
        id = f"{video_info_dict['id']}{sub_video_counter:04}"
        sub_video_label['annotations']['meta']['task']['id'] = id
        sub_video_label['annotations']['meta']['task']['name'] = f"{sub_video_file.name}" 
        sub_video_label['annotations']['meta']['task']['size'] = end_frame - start_frame 
        sub_video_label['annotations']['meta']['task']['start_frame'] = start_frame 
        sub_video_label['annotations']['meta']['task']['stop_frame'] = end_frame 
        sub_video_label['annotations']['meta']['task']['segments'] = OrderedDict({'segment': OrderedDict({'id': video_info_dict['segments']['segment']['id'], 'start': start_frame, 'stop': end_frame, 'url': video_info_dict['segments']['segment']['url']})}) 
        try:
            track_list = split_tracks(deepcopy(label_xml_dict['annotations']['track']), start_frame, end_frame)
            if len(track_list) > 0:
                sub_video_label['annotations']['track'] = track_list
            else:
                del sub_video_label['annotations']['track']
        except KeyError:
            pass
            
        with open(f"{str(label_out_path)}/{id}.xml", 'w') as f:
            xmltodict.unparse(sub_video_label, f)

        # increment for next iter
        sub_video_counter +=1

[INFO] 11/04/2022 01:00:39PM: MainProcess: 257686485.py - Splitting video: 48
[INFO] 11/04/2022 01:00:47PM: MainProcess: 257686485.py - Creating sub-video: 48/0000


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (768, 1080) to (768, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[INFO] 11/04/2022 01:00:49PM: MainProcess: 257686485.py - Creating sub-video: 48/0001


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (768, 1080) to (768, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[INFO] 11/04/2022 01:00:51PM: MainProcess: 257686485.py - Creating sub-video: 48/0002


...

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1024, 1528) to (1024, 1536) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[INFO] 11/04/2022 01:32:56PM: MainProcess: 257686485.py - Creating sub-video: 46/0019


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1024, 1528) to (1024, 1536) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


[INFO] 11/04/2022 01:33:00PM: MainProcess: 257686485.py - Creating sub-video: 46/0020


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1024, 1528) to (1024, 1536) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
